In [1]:
import os
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


C:\Users\KaiLo\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset sin informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que NO contengan nubes. Se plantea este escenario con el fin de determinar el rendmiento del modelo con un problema de este estilo.

In [2]:
path = './satellite_dataset-sin-nubes.csv'

satellite = pd.read_csv(path)
satellite.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,272.0,172.0,74.0,61.0,81.0,62.0,1,0,0,0
1,172.0,158.0,71.0,62.0,107.0,92.0,1,0,0,0
2,172.0,104.0,14.0,1.0,44.0,39.0,1,0,0,0
3,255.0,202.0,103.0,87.0,128.0,104.0,1,0,0,0
4,255.0,197.0,100.0,86.0,113.0,92.0,1,0,0,0


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [9]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,label
0,272.0,172.0,74.0,61.0,81.0,62.0,wofs,NaN,NaN,NaN,wofs
1,172.0,158.0,71.0,62.0,107.0,92.0,wofs,NaN,NaN,NaN,wofs
2,172.0,104.0,14.0,1.0,44.0,39.0,wofs,NaN,NaN,NaN,wofs
3,255.0,202.0,103.0,87.0,128.0,104.0,wofs,NaN,NaN,NaN,wofs
4,255.0,197.0,100.0,86.0,113.0,92.0,wofs,NaN,NaN,NaN,wofs
5,139.0,341.0,185.0,2804.0,1263.0,464.0,NaN,bosque,NaN,NaN,bosque
6,199.0,112.0,16.0,5.0,47.0,39.0,wofs,NaN,NaN,NaN,wofs
7,251.0,266.0,53.0,-4.0,35.0,32.0,wofs,NaN,NaN,NaN,wofs
8,736.0,702.0,623.0,627.0,667.0,540.0,NaN,NaN,NaN,ninguno,ninguno
9,414.0,484.0,286.0,240.0,246.0,199.0,wofs,NaN,NaN,NaN,wofs


In [10]:
# Se crea el conjunto de calsificacion y el de entrenamiento
Y = satellite["label"]
# X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","ninguno"], axis=1)
Y.value_counts()

wofs           753845
bosque         201821
ninguno         42344
wofs_bosque      1990
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [11]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [12]:
X_test

,blue,green,red,nir,swir1,swir2
157105,212.0,186.0,103.0,98.0,146.0,119.0
374554,247.0,319.0,185.0,155.0,199.0,168.0
688694,133.0,155.0,53.0,880.0,244.0,91.0
265381,334.0,296.0,128.0,79.0,78.0,68.0
955415,16.0,290.0,142.0,3104.0,1311.0,518.0
4280,136.0,347.0,187.0,2608.0,1149.0,413.0
514674,142.0,425.0,216.0,3270.0,1320.0,439.0
800352,328.0,351.0,121.0,60.0,72.0,55.0
94727,286.0,226.0,111.0,95.0,116.0,96.0
753889,242.0,523.0,323.0,3661.0,1737.0,679.0


In [13]:
y_test

157105           wofs
374554           wofs
688694         bosque
265381           wofs
955415         bosque
4280           bosque
514674         bosque
800352           wofs
94727            wofs
753889         bosque
334658           wofs
676584           wofs
796710           wofs
385008           wofs
568106           wofs
25532     wofs_bosque
840001           wofs
726752           wofs
939105         bosque
608171           wofs
58761            wofs
139219           wofs
711187           wofs
130290           wofs
91739            wofs
853663           wofs
436355         bosque
816093           wofs
580080           wofs
916799        ninguno
             ...     
916413           wofs
67988          bosque
209129        ninguno
669902           wofs
896462           wofs
867843           wofs
506360           wofs
600883           wofs
527117         bosque
228533           wofs
819510        ninguno
18465            wofs
770866        ninguno
314048         bosque
452967    

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

In [14]:
#Create a Gaussian Classifier
gnb = GaussianNB()
#gnb = MultinomialNB()

In [15]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [16]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [17]:
Counter(y_pred)

Counter({'wofs': 224587,
         'wofs_bosque': 951,
         'bosque': 56310,
         'ninguno': 18152})

In [18]:
confusion_matrix(y_test, y_pred)

array([[ 55844,   2138,   1898,    629],
       [   398,   8423,   3624,    268],
       [    45,   7539, 218536,     34],
       [    23,     52,    529,     20]], dtype=int64)

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.92      0.96     60509
     ninguno       0.46      0.66      0.55     12713
        wofs       0.97      0.97      0.97    226154
 wofs_bosque       0.02      0.03      0.03       624

    accuracy                           0.94    300000
   macro avg       0.61      0.65      0.62    300000
weighted avg       0.95      0.94      0.95    300000



In [20]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9427433333333334


In [21]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [22]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
#joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

## Conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 98.50% y el rendimiento con el modelo Gausiano alcanza el 95.92%. Por lo que logramos determinar que el modelo Multinomial logra funcionar mejor por 3 puntos de diferencia. Revisando la matriz de confusión logramos determinar que la principal diferencia entre los modelos es que en el multinomial se confunde menos entre elementos de la clase 2(Ninguno) y la clase 3(WOFS - Agua).

**MultinomialGNB**

Accuracy: 0.9850353183662661

array([[ 57498,    235,      0,     80],
       [   803,   6680,   1210,     84],
       [     0,   1160, 210992,    576],
       [     1,      8,     25,    106]], dtype=int64)

**GaussianMB**

Accuracy: 0.9592818956694745

array([[ 54306,   1897,     13,   1597],
       [   320,   8068,    160,    229],
       [     8,   5578, 205658,   1484],
       [     6,     23,     64,     47]], dtype=int64)

# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra